modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/code_agents.ipynb

In [1]:
import base64
import datetime
import os
import time

import numpy as np
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    FinalAnswerTool,
    Tool,
    VisitWebpageTool,
    tool,
)
from utils import gemini_model_OpenAIServer
from utils import gemini_model_liteLLM

In [2]:
USERNAME = "CoralLeiCN"

In [3]:
model = gemini_model_OpenAIServer("gemini-2.0-flash")

## Selecting a Playlist for the Party Using `smolagents`

In [4]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)
agent.run(
    "Search for the best music recommendations for a party at the Wayne's mansion."
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_query = "elegant party music recommendations Wayne mansion"                                               
  search_results = web_search(query=search_query)                                                                  
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Top Songs - Mansion Accompaniment Tracks - 
YouTube](https://www.youtube.com/playlist?list=OLAK5uy_lc2iMZFGTt284HeUy_JL-lgllrnragSsw)
Share your videos with friends, family, and the world

[Main Character Energy: Perfect Party Entrance Songs | 
TikTok](https://www.tiktok.com/@elegantmusicgroup/video/7464997986632879406)
249 Likes, TikTok video from Elegant Music Group (@elegantmusicgroup): "Discover the best songs to enhance your 
grand entrance and embody Main Character Energy at your next elegant party. ... Keywords: best entrance songs for 
parties, elegant party music suggestions, grand entrance songs for events, main character energy playlist, songs 
for a ...

[Party on Wayne - playlist by Music League | Spotify](https://open.spotify.com/playlist/66wXTng3HRUePKf9WdOrrK)
Party on Wayne · Playlist · 20 songs · 6 likes. Party on Wayne · Playlist · 20 songs · 6 likes. Party on Wayne · 
Playlist · 20 songs · 6 likes. Home; Search; Your Library. Create your first playlist It's easy, we'll help you. 
Create playlist. Let's find some ...

[Beautiful Wedding Instrumental Playlist | Romantic & Elegant Music for 
...](https://m.youtube.com/watch?v=afqXQxvHlKI)
Create the perfect atmosphere for your wedding with this elegant instrumental playlist, featuring timeless melodies
that capture the romance and joy of your ...

[Classical Music Playlist for Weddings, Dinner Parties & Entertaining 
...](https://m.youtube.com/watch?v=aLiwCrlcjSQ)
A Classical Music Playlist perfect for your dinner party. Nice and relaxing. Music available at 
www.soundings.comFor more information, please visit soundin...

[Event Party Venue in Savannah, Vacation Rental | The Wayne Mansion](https://www.thewaynemansion.com/)
Learn more about The Wayne Mansion. G allery. View Photos. 204 E Hall St Savannah, GA 31401. Tel: 912-257-4050. 
Email: staylucky@luckysavannah.com. Book .

[Free Elegant Music - Uppbeat](https://uppbeat.io/music/category/elegant)
70 free elegant tracks. Download the best royalty free elegant music for YouTube, Twitch, Instagram, TikTok, 
podcasts and more. Uppbeat is the free music platform for creators.

[Dinner Party Tips From Designer Lorna Gross - Mansion 
Global](https://www.mansionglobal.com/boutique/articles/curating-an-elegant-dinner-party-with-help-from-designer-lo
rna-gross-67b28ace)
Using her entertaining experience as inspiration, Gross shared some of her favorite items from Mansion Global 
Boutique to curate an elegant dinner party at home. Guest Curator Set of 6 Gold-Plated ...

[The Wayne Mansion - Lucky Savannah](https://www.luckysavannah.com/vacation-rentals/wayne-mansion)
The Wayne Mansion is on one of Savannah's most well-known streets, surrounded by stunning renovated mansions, just 
steps from Forysth Park. Indulge in the history of Hall Street, while being centrally located to everything in the 
Historic District. This 9 bedroom estate can accommodate up to 18 Lucky guests, and boasts nearly 5,000 square 
feet.

[MUSIC AT THE MANSION | conrad-caldwell](https://www.conradcaldwell.org/music-at-the-mansion)
Enter the Gilded Age ambiance with "Music at the Mansion," a sophisticated salon series. Enjoy one of two (or 
both!) exclusive chamber quartet performances that transport you to the refined gatherings of that era. ... $50.00 
per person, per performance, indulge in classical masterpieces, champagne, and gourmet appetizers, capturing the 
elegant ...

Out: None

[Step 1: Duration 1.81 seconds| Input tokens: 2,249 | Output tokens: 116]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("For a Wayne's mansion party, I recommend a mix of classical, jazz, and upbeat music. Consider      
  playlists like 'Classical Music Playlist for Weddings, Dinner Parties & Entertaining' for ambiance, 'Jazz Music  
  Upbeat | Best Upbeat Jazz Songs' or 'Best Upbeat Jazz Songs - Positive Energy Jazz Music' for a lively           
  atmosphere. Explore Elegant Music Group's website for DJ mixes, band performances, or their 'Main Character      
  Energy: Perfect Party Entrance Songs' TikTok playlist. Additionally, 'Exquisite Mood Smooth Jazz - Relax         
  Elegant Jazz Music for Coffee Break' could be a good addition for a relaxed and sophisticated vibe.")            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: For a Wayne's mansion party, I recommend a mix of classical, jazz, and upbeat music. Consider 
playlists like 'Classical Music Playlist for Weddings, Dinner Parties & Entertaining' for ambiance, 'Jazz Music 
Upbeat | Best Upbeat Jazz Songs' or 'Best Upbeat Jazz Songs - Positive Energy Jazz Music' for a lively atmosphere. 
Explore Elegant Music Group's website for DJ mixes, band performances, or their 'Main Character Energy: Perfect 
Party Entrance Songs' TikTok playlist. Additionally, 'Exquisite Mood Smooth Jazz - Relax Elegant Jazz Music for 
Coffee Break' could be a good addition for a relaxed and sophisticated vibe.

[Step 4: Duration 2.07 seconds| Input tokens: 15,900 | Output tokens: 731]

"For a Wayne's mansion party, I recommend a mix of classical, jazz, and upbeat music. Consider playlists like 'Classical Music Playlist for Weddings, Dinner Parties & Entertaining' for ambiance, 'Jazz Music Upbeat | Best Upbeat Jazz Songs' or 'Best Upbeat Jazz Songs - Positive Energy Jazz Music' for a lively atmosphere. Explore Elegant Music Group's website for DJ mixes, band performances, or their 'Main Character Energy: Perfect Party Entrance Songs' TikTok playlist. Additionally, 'Exquisite Mood Smooth Jazz - Relax Elegant Jazz Music for Coffee Break' could be a good addition for a relaxed and sophisticated vibe."

## Using a Custom Tool to Prepare the Menu

In [7]:
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=model)

In [8]:
agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  occasion = input("What is the occasion for the party?")                                                          
  print(occasion)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'occasion = input("What is the occasion for the party?")' due to: InterpreterError: 
It is not permitted to evaluate other functions than the provided tools or functions defined/imported in previous 
code (tried to execute input).

[Step 1: Duration 0.83 seconds| Input tokens: 2,228 | Output tokens: 76]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
**Formal Party Menu**

**Appetizer:** Smoked Salmon Canapés with Dill Cream Cheese

**Soup:** Creamy Tomato Soup with Basil

**Main Course:** Roasted Beef Tenderloin with Red Wine Reduction, served with Asparagus and Garlic Mashed Potatoes

**Dessert:** Chocolate Lava Cake with Raspberry Coulis and Vanilla Ice Cream

[Step 5: Duration 0.77 seconds| Input tokens: 13,168 | Output tokens: 397]

'\n**Formal Party Menu**\n\n**Appetizer:** Smoked Salmon Canapés with Dill Cream Cheese\n\n**Soup:** Creamy Tomato Soup with Basil\n\n**Main Course:** Roasted Beef Tenderloin with Red Wine Reduction, served with Asparagus and Garlic Mashed Potatoes\n\n**Dessert:** Chocolate Lava Cake with Raspberry Coulis and Vanilla Ice Cream\n'

## Using Python Imports Inside the Agent

In [9]:
agent = CodeAgent(tools=[], model=model, additional_authorized_imports=["datetime"])
agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  drinks = 30                                                                                                      
  mansion = 60                                                                                                     
  menu = 45                                                                                                        
  music = 45                                                                                                       
                                                                                                                   
  total_time = drinks + mansion + menu + music                                                                     
  print(total_time)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
180

Out: None

[Step 1: Duration 1.19 seconds| Input tokens: 2,253 | Output tokens: 109]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(answer="2025-03-22 18:52:24")                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2025-03-22 18:52:24

[Step 3: Duration 0.50 seconds| Input tokens: 7,477 | Output tokens: 288]

'2025-03-22 18:52:24'

## Sharing Our Custom Party Preparator Agent to the Hub

In [10]:
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets.",
        }

        return themes.get(
            category.lower(),
            "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.",
        )


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
    ],
    model=model,
    max_steps=10,
    verbosity_level=2,
)

In [11]:
agent.run(
    "Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to suggest a playlist for a villain masquerade party at Wayne's mansion. This means the music      
should be fitting for a sophisticated masquerade ball but with a slightly dark and villainous undertone. I'll start
by searching for suitable music genres and themes.                                                                 
Code:                                                                                                              
```py                                                                                                              
print(web_search(query="masquerade ball music playlist"))                                                          
print(web_search(query="villain themed party music playlist"))                                                     
print(web_search(query="dark classical music playlist"))                                                           
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print(web_search(query="masquerade ball music playlist"))                                                        
  print(web_search(query="villain themed party music playlist"))                                                   
  print(web_search(query="dark classical music playlist"))                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The Complete playlist for your next Masquerade 
party](https://vivomasks.com/blogs/masquerade/the-complete-playlist-for-your-next-masquerade-party)
The Masquerade Waltz by Armenian composer Aram Khachaturian is a must-add on your masquerade party playlist. 
Originally written to accompany a play, the Masquerade Suite written by Mikhail Lermontov. Once you hear this 
music, you can already imagine the highs and lows of emotion that one may encounter while at a masquerade ball. The
heightened sensations evoked by the wildness of the music ...

[Masquerade Ball Music](https://music.youtube.com/playlist?list=PLmfMvyItKJ0pPO3EjgXTwbiIJZWT23Sl-)
With the YouTube Music app, enjoy over 100 million songs at your fingertips, plus albums, playlists, remixes, music
videos, live performances, covers, and hard-to-find music you can't get anywhere else.

[Songs to play at a masquerade ball - 
YouTube](https://www.youtube.com/playlist?list=PLDAD3zW4NbMiYq371qad7WS78MDCIzM2z)
Share your videos with friends, family, and the world

[Classical music for masquerade ball - playlist by jamie | 
Spotify](https://open.spotify.com/playlist/3ucte5LOvis88JoEOap6aL)
Classical music for masquerade ball · Playlist · 27 songs · 53 likes

[masquerade ball a playlist - playlist by sophia:p | 
Spotify](https://open.spotify.com/playlist/3Bh77Is7zdjTS5r2qCsCeJ)
masquerade ball a playlist · Playlist · 24 songs · 313 likes

[Masquerade ball music - playlist by Jere T Weliver | 
Spotify](https://open.spotify.com/playlist/2F5ihZRdsUQqHVHw6NMiA3)
Playlist · Masquerade ball music · 20 songs · 32 likes

[Beautiful Waltz Music - Masquerade Ball [2 Hour Version]](https://www.youtube.com/watch?v=cdI9ZexmCQs)
*** Beautiful waltz music about a masquerade ball, where people dress up as knights, queens, princes, archers, and 
even fantasy creatures. This music I composed is called Masquerade Ball.

[classical music for a masquerade ball | royalty core playlist](https://www.youtube.com/watch?v=uG1Np9zWzbk)
i g n o r e m e (tags): study , playlist , studying , piano , relaxing , sleeping , romantic , chill , aesthetic , 
classical music , princess , pov, ambience, ambient , masquerade ball , waltz ...

[My masquerade ball music for balls and 
dancing](https://music.youtube.com/playlist?list=PLK9I-WraRk1ixaWJNgRV8qBW8NBlvgUVD)
A new music service with official albums, singles, videos, remixes, live performances and more for Android, iOS and
desktop. It's all here.

[Stream DJ Nadz | Listen to Masquerade Ball playlist online for free on 
...](https://soundcloud.com/nadia-juarez-cuamatzi/sets/masquerade-ball)
Listen to Masquerade Ball, a playlist curated by DJ Nadz on desktop and mobile.
## Search Results

[The 99 Best Songs for Villains (Our Villain Arc Playlist) - 
iSpyTunes](https://www.ispytunes.com/post/best-songs-for-villains)
Enter the captivating world of villain songs with our curated playlist of the 99 best songs for villains, where 
darkness and power converge. ... Its slow, distorted beats and eerie atmospheres create a chilling ambiance that 
aligns with the dark themes often associated with villains. Hardstyle, on the other hand, is a high-energy genre 
...

[Villain Mode - 100 Songs - playlist by Spotify](https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH)
Playlist · Villain Mode · 100 items · 1.4M saves. Playlist · Spotify · 100 items · 1.4M saves. Playlist · Villain 
Mode · 100 items · 1.4M saves. Playlist · Spotify · 100 items · 1.4M saves. What do you want to play? CtrlK. 
Premium Support Download. Install App. Sign up Log in. Resize main navigation ...

[ViLLAiN THEMES - this dark playlist will make you feel like a epic 
...](https://open.spotify.com/playlist/4lYzE6v91D5g66J17wbmum)
ViLLAiN THEMES - this dark playlist will make you feel like a epic villain · Playlist · 157 songs · 726 likes

[Villain playlists - YouTube Music](https://music.youtube.com/playlist?list=PLo66hThdCl6GGI6NwYbcNpk8pdP6v8AiY)
With the YouTube Mu

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the search results, I can compile a playlist that incorporates a mix of classical pieces with    
dark themes, orchestral villain themes, and some modern interpretations. Here's a potential playlist structure:    
                                                                                                                   
1.  **Overture/Opening:** Start with a grand and ominous piece, like "Toccata and Fugue in D Minor" by Bach.       
2.  **Waltz/Ballroom:** Transition into waltzes with a darker feel, such as selections from Khachaturian's         
"Masquerade Suite."                                                                                                
3.  **Scheming/Plotting:** Include pieces that evoke a sense of mystery and intrigue, like excerpts from           
Beethoven's "Moonlight Sonata" or Mozart's "Requiem."                                                              
4.  **Villain Theme Songs:** Incorporate orchestral themes from villainous characters in movies, such as Darth     
Vader's theme ("The Imperial March") or Maleficent's theme.                                                        
5.  **Climax/Confrontation:** Build up to more intense and dramatic pieces, such as selections from Carl Orff's    
"Carmina Burana" or Shostakovich's "Symphony No. 5."                                                               
6.  **Resolution/Aftermath:** Conclude with a somber and reflective piece, like Chopin's "Funeral March."          
                                                                                                                   
I will formulate a prompt for the final answer.                                                                    
Code:                                                                                                              
```py                                                                                                              
final_answer("""Here's a playlist for a villain masquerade party at Wayne's mansion:                               
                                                                                                                   
1.  Overture: Bach - Toccata and Fugue in D Minor                                                                  
2.  Waltz: Khachaturian - Masquerade Suite (selections)                                                            
3.  Scheming: Beethoven - Moonlight Sonata (1st Movement), Mozart - Requiem (Lacrimosa)                            
4.  Villain Themes: Star Wars - The Imperial March (Darth Vader's Theme), Maleficent's theme                       
5.  Climax: Carl Orff - Carmina Burana (O Fortuna), Shostakovich - Symphony No. 5 (selections)                     
6.  Resolution: Chopin - Funeral March                                                                             
                                                                                                                   
This playlist blends classical music with villainous themes, perfect for a sophisticated yet dark masquerade       


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("""Here's a playlist for a villain masquerade party at Wayne's mansion:                             
                                                                                                                   
  1.  Overture: Bach - Toccata and Fugue in D Minor                                                                
  2.  Waltz: Khachaturian - Masquerade Suite (selections)                                                          
  3.  Scheming: Beethoven - Moonlight Sonata (1st Movement), Mozart - Requiem (Lacrimosa)                          
  4.  Villain Themes: Star Wars - The Imperial March (Darth Vader's Theme), Maleficent's theme                     
  5.  Climax: Carl Orff - Carmina Burana (O Fortuna), Shostakovich - Symphony No. 5 (selections)                   
  6.  Resolution: Chopin - Funeral March                                                                           
                                                                                                                   
  This playlist blends classical music with villainous themes, perfect for a sophisticated yet dark masquerade     
  ball.""")                                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Here's a playlist for a villain masquerade party at Wayne's mansion:

1.  Overture: Bach - Toccata and Fugue in D Minor
2.  Waltz: Khachaturian - Masquerade Suite (selections)
3.  Scheming: Beethoven - Moonlight Sonata (1st Movement), Mozart - Requiem (Lacrimosa)
4.  Villain Themes: Star Wars - The Imperial March (Darth Vader's Theme), Maleficent's theme
5.  Climax: Carl Orff - Carmina Burana (O Fortuna), Shostakovich - Symphony No. 5 (selections)
6.  Resolution: Chopin - Funeral March

This playlist blends classical music with villainous themes, perfect for a sophisticated yet dark masquerade ball.

[Step 3: Duration 3.04 seconds| Input tokens: 15,783 | Output tokens: 653]

"Here's a playlist for a villain masquerade party at Wayne's mansion:\n\n1.  Overture: Bach - Toccata and Fugue in D Minor\n2.  Waltz: Khachaturian - Masquerade Suite (selections)\n3.  Scheming: Beethoven - Moonlight Sonata (1st Movement), Mozart - Requiem (Lacrimosa)\n4.  Villain Themes: Star Wars - The Imperial March (Darth Vader's Theme), Maleficent's theme\n5.  Climax: Carl Orff - Carmina Burana (O Fortuna), Shostakovich - Symphony No. 5 (selections)\n6.  Resolution: Chopin - Funeral March\n\nThis playlist blends classical music with villainous themes, perfect for a sophisticated yet dark masquerade ball."

In [ ]:
# agent.push_to_hub(f'{USERNAME}/AlfredAgent')

In [14]:
agent = CodeAgent(tools=[], model=gemini_model_liteLLM("gemini-2.0-flash"))
alfred_agent = agent.from_hub(f"{USERNAME}/AlfredAgent", trust_remote_code=True)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [15]:
alfred_agent.model = model

In [16]:
alfred_agent.run(
    "Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to generate a playlist for a villain masquerade party at Wayne's mansion. This implies a           
sophisticated, perhaps slightly dark, and definitely thematic musical selection. I will use web search to find     
suitable playlists or songs that fit the theme.                                                                    
Code:                                                                                                              
```py                                                                                                              
playlists = web_search(query="villain masquerade party playlist")                                                  
print(playlists)                                                                                                   
songs = web_search(query="songs for a masquerade ball")                                                            
print(songs)                                                                                                       
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlists = web_search(query="villain masquerade party playlist")                                                
  print(playlists)                                                                                                 
  songs = web_search(query="songs for a masquerade ball")                                                          
  print(songs)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[you're at a masquerade and unknowingly dancing with the 
villain](https://open.spotify.com/playlist/4zxYvQs9gByjRhAn5ujSfQ)
you're at a masquerade and unknowingly dancing with the villain · Playlist · 89 songs · 10 likes

[dancing with the villain during a masquerade ball - playlist ... - 
Spotify](https://open.spotify.com/playlist/17SEJ9K7DyEAhgoJgZlyNN)
mkisay · Playlist · 40 songs · 33 likes

[Masquerade party music - playlist by drew - Spotify](https://open.spotify.com/playlist/3gICKSayCeN6muUWNGHeK4)
Playlist · Masquerade party music 🎭 · 38 items · 8 saves

[a stormy night, an ancient castle, and a vampire masquerade (dark 
...](https://www.youtube.com/watch?v=i9yqAmS5DiY)
⋅ • ☾ Travelling through the woods one dark evening, you are caught in a storm, but fortunately, an abandoned 
palace looms ahead. Entering, you find myster...

[Google 
Colab](https://colab.research.google.com/github/huggingface/agents-course/blob/main/notebooks/unit2/smolagents/code
_agents.ipynb)
However, I will focus on the "Masquerade Party! - playlist by freli" as it has more songs and higher popularity. To
finalize the playlist, I will visit the Spotify page of "Masquerade Party! - playlist by freli" to get the list of 
songs and ensure they fit the villain masquerade theme well.

[Villain Scarlet Witch Era - A Playlist - YouTube](https://www.youtube.com/watch?v=mqrJEqMPcvk)
"You Break The Rules And Become A Hero. I Do It And I Become The Enemy."0:00 Siouxxie - Masquerade2:26 CK9C 
[ChaoticCanineCulture] - Run Run!4:35 Crywolf - A...

[Songs You Need on Your Villain Era Playlist - 
ArtieCarden](https://artiecarden.com/2023/04/10/villain-era-playlist-songs/)
3 thoughts on " Songs You Need on Your Villain Era Playlist " Artiecarden says: September 29, 2024 at 6:40 pm. 
you're welcome, i'm glad it helped 🙂 . Like ... 2024 at 3:38 am. Great list! Compiling music for mine and my 
husband's Addams Family Themed vow renewal and party, and also for a Halloween bellydance performance as villain 
...

[Masquerade Party - DJMone - Serato DJ Playlists](https://serato.com/playlists/DJMone/masquerade-party)
Masquerade Party Serato Playlist by DJMone. Serato Playlists. DJMone. Masquerade Party. Tracklist. 9:08pm 00:00:00 
Miguel - Quickie (Remix) (Clean) 00:06:38 Solange ft Snoop Dogg - I Decided (Remix) (Clean) 00:11:30 Usher - Climax
(Intro Outro) 00:12:03 Brandy ft Chris Brown - Put It Down (Clean) ...

[Masquerade Party Mix - YouTube](https://www.youtube.com/watch?v=mWaboGALcbA)
A mix of Disco House, Nu Disco, a few old school classics and a little Deep House for that Masquerade 
Party.DISCLAIMER: If you are experiencing ads during a...

[Songs That Make You Feel Like a Villain - Playlist 
Ideas](https://playlistideas.com/songs-that-make-you-feel-like-a-villain/)
Music has a unique ability to transport us to different emotional landscapes, and this playlist is designed to make
you feel like the villain you secretly want to be. Whether you're plotting schemes, seeking revenge, or simply 
reveling in your darker side, these songs will accompany you on your journey into the realms of villainy.
## Search Results

[masquerade ball a playlist - playlist by sophia:p | 
Spotify](https://open.spotify.com/playlist/3Bh77Is7zdjTS5r2qCsCeJ)
masquerade ball a playlist · Playlist · 24 songs · 313 likes

[Songs to play at a masquerade ball - 
YouTube](https://www.youtube.com/playlist?list=PLDAD3zW4NbMiYq371qad7WS78MDCIzM2z)
Share your videos with friends, family, and the world

[classical music for a masquerade ball | royalty core playlist](https://www.youtube.com/watch?v=uG1Np9zWzbk)
i g n o r e m e (tags): study , playlist , studying , piano , relaxing , sleeping , romantic , chill , aesthetic , 
classical music , princess , pov, ambience, ambient , masquerade ball , waltz ...

[Masquerade Party! - playlist by freli - Spotify](https://open.spotify.com/playlist/6P6sYHRI3ERiSOG8ljaeFz)
Masquerade Party!🎭 · Playlist · 171 songs · 21 likes

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I have results for Dark Classical and Dark Pop playlists. I will now generate a final playlist by         
combining songs from all the searches performed so far.                                                            
                                                                                                                   
**Core Villain Masquerade Playlist**                                                                               
                                                                                                                   
1.  "Masquerade" (Siouxxie)                                                                                        
2.  "Run Run!" (CK9C)                                                                                              
3.  "Aram Khachaturian - Masquerade Suite - Waltz"                                                                 
4.  "Sympathy for the Devil" (The Rolling Stones)                                                                  
5.  "Heathens" (Twenty One Pilots)                                                                                 
6.  "Bad Guy" (Billie Eilish)                                                                                      
7.  "I Did Something Bad" (Taylor Swift)                                                                           
                                                                                                                   
**Dark Classical Additions:**                                                                                      
                                                                                                                   
*   Based on the search results, I can add pieces by Mozart, Bach, and Shostakovich to the playlist. Some specific 
tracks:                                                                                                            
    *   Mozart - Requiem K. 626 - Lacrimosa                                                                        
    *   Bach - Toccata and Fugue in D Minor, BWV 565                                                               
    *   Shostakovich - Symphony No. 5                                                                              
                                                                                                                   
**Dark Pop Additions:**                                                                                            
                                                                                                                   
*   Based on the search results, I can add songs from Billie Eilish, Halsey, and Lana Del Rey. A few specific      
suggestions:                                                                                                       
    *   Billie Eilish - "you should see me in a crown"                                                             
    *   Halsey - "Control"                                                                                         
    *   Lana Del Rey - [0

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_playlist = """                                                                                             
  Wayne Manor Villain Masquerade Playlist:                                                                         
                                                                                                                   
  Core Villain Masquerade:                                                                                         
  1. Masquerade (Siouxxie)                                                                                         
  2. Run Run! (CK9C)                                                                                               
  3. Aram Khachaturian - Masquerade Suite - Waltz                                                                  
  4. Sympathy for the Devil (The Rolling Stones)                                                                   
  5. Heathens (Twenty One Pilots)                                                                                  
  6. Bad Guy (Billie Eilish)                                                                                       
  7. I Did Something Bad (Taylor Swift)                                                                            
                                                                                                                   
  Dark Classical:                                                                                                  
  8. Mozart - Requiem K. 626 - Lacrimosa                                                                           
  9. Bach - Toccata and Fugue in D Minor, BWV 565                                                                  
  10. Shostakovich - Symphony No. 5                                                                                
                                                                                                                   
  Dark Pop:                                                                                                        
  11. Billie Eilish - you should see me in a crown                                                                 
  12. Halsey - Control                                                                                             
  13. Lana Del Rey - Born to Die                                                                                   
  14. The Cranberries - Zombie                                                                                     
  """                                                                                                              
  final_answer(final_playlist)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Wayne Manor Villain Masquerade Playlist:

Core Villain Masquerade:
1. Masquerade (Siouxxie)
2. Run Run! (CK9C)
3. Aram Khachaturian - Masquerade Suite - Waltz
4. Sympathy for the Devil (The Rolling Stones)
5. Heathens (Twenty One Pilots)
6. Bad Guy (Billie Eilish)
7. I Did Something Bad (Taylor Swift)

Dark Classical:
8. Mozart - Requiem K. 626 - Lacrimosa
9. Bach - Toccata and Fugue in D Minor, BWV 565
10. Shostakovich - Symphony No. 5

Dark Pop:
11. Billie Eilish - you should see me in a crown
12. Halsey - Control
13. Lana Del Rey - Born to Die
14. The Cranberries - Zombie

[Step 4: Duration 3.21 seconds| Input tokens: 21,072 | Output tokens: 212]

'\nWayne Manor Villain Masquerade Playlist:\n\nCore Villain Masquerade:\n1. Masquerade (Siouxxie)\n2. Run Run! (CK9C)\n3. Aram Khachaturian - Masquerade Suite - Waltz\n4. Sympathy for the Devil (The Rolling Stones)\n5. Heathens (Twenty One Pilots)\n6. Bad Guy (Billie Eilish)\n7. I Did Something Bad (Taylor Swift)\n\nDark Classical:\n8. Mozart - Requiem K. 626 - Lacrimosa\n9. Bach - Toccata and Fugue in D Minor, BWV 565\n10. Shostakovich - Symphony No. 5\n\nDark Pop:\n11. Billie Eilish - you should see me in a crown\n12. Halsey - Control\n13. Lana Del Rey - Born to Die\n14. The Cranberries - Zombie\n'

## Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

In [4]:
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_AUTH = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    "https://cloud.langfuse.com/api/public/otel"  # EU data region
)
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [5]:
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [6]:
agent = CodeAgent(tools=[], model=model)
alfred_agent = agent.from_hub(f"{USERNAME}/AlfredAgent", trust_remote_code=True)
alfred_agent.model = model

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
alfred_agent.run(
    "Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Since this is a 'villain masquerade' party at Wayne's mansion, the music should reflect a dark,           
mysterious, and perhaps sophisticated vibe, fitting for both villains and a masquerade ball. I will use the        
web_search tool to find playlists suitable for a villain masquerade party, considering the setting of Wayne's      
mansion. I will then analyze the results and create a playlist based on the themes and suggestions.                
Code:                                                                                                              
```py                                                                                                              
search_results = web_search(query="playlist for villain masquerade party batman gotham")                           
print(search_results)                                                                                              
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="playlist for villain masquerade party batman gotham")                         
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Google 
Colab](https://colab.research.google.com/github/huggingface/agents-course/blob/main/notebooks/unit2/smolagents/code
_agents.ipynb)
However, I will focus on the "Masquerade Party! - playlist by freli" as it has more songs and higher popularity. To
finalize the playlist, I will visit the Spotify page of "Masquerade Party! - playlist by freli" to get the list of 
songs and ensure they fit the villain masquerade theme well.

[Category:Gotham Villains - Fandom](https://villains.fandom.com/wiki/Category:Gotham_Villains)
Villains who appear in the DC Television series Gotham, a TV series that is based off the Batman franchise by DC 
Comics.. NOTE: As the series takes place before Bruce Wayne officially becomes Batman do not add any to Batman's 
Villains category wise as that is for villains who officially fight Batman. Being The Joker, The Penguin, and The 
Riddler the only exception, since they were the only ...

[04: The Masquerade - Gotham Knights Guide - IGN](https://www.ign.com/wikis/gotham-knights/04:_The_Masquerade)
This Gotham Knights walkthrough will outline both of The Masquerade's missions and detail how to solve the puzzles 
and beat down the villains in this section so you can continue your progress ...

[构建使用代码的智能体 - Hugging Face Agents 
Course](https://huggingface.co/learn/agents-course/zh-CN/unit2/smolagents/code_agents)
The party idea is a 'villain masquerade' theme") 如你所见，我们创建了一个具有多种工具的 CodeAgent 
，这些工具增强了智能体的功能，将其变成了准备好与社区分享的终极派对策划者!

[Gotham Knights Walkthrough - Part 8 - The Masquerade [4K 60fps]](https://www.youtube.com/watch?v=mfhJaX6Cs-g)
Gotham Knights Story Walkthrough Case File 4.1 "The Masquerade: Mark Hendricks" and Case File 4.2 "The Masquerade: 
The Orchard Hotel" (as Robin) [PC 4K @ 60 ...

[44 adult Gotham party ideas | batman party, superhero party ... - 
Pinterest](https://www.pinterest.com/cynabunz230/adult-gotham-party-ideas/)
Jul 2, 2020 - Explore Cynthia Perry's board "adult Gotham party ideas", followed by 179 people on Pinterest. See 
more ideas about batman party, superhero party, party.

[The Gotham Costume Party - Cavanaugh Stories](https://www.cavanaughstories.com/post/the-gotham-costume-party)
Batman and Robin had been chasing a brand new villain through Gotham City on Halloween night when they trailed him 
into what appeared to be a costume party being held by a bunch of social media influencers. They knew very little 
of the villain other than that he had some sort of transformative ability which most certainly spelled trouble for 
the citizens of Gotham. Multiple people had already ...

[Batman Birthday Party: 9 Ideas and Tips for a Super 
Celebration](https://www.bagvania.com/batman-birthday-party-9-ideas-and-tips/)
Batman Birthday Party - Costume Idea. Credit: Sum.ae 4. Games and Activities: Bring Out the Heroic Fun. Games and 
activities are often the best ideas to keep everyone engaged and entertained, so don't skip this step to make sure 
your Batman birthday party is a success. Keep guests entertained with Batman-themed games that test their skills 
and imagination.

[Category:Batman: Gotham Knight Villains | Batman Wiki | 
Fandom](https://batman.fandom.com/wiki/Category:Batman:_Gotham_Knight_Villains)
Villains appearing in Batman: Gotham Knight. Fandom Apps Take your favorite fandoms with you and never miss a beat.

[Building Agents That Use Code - Hugging Face Agents 
Course](https://huggingface.co/learn/agents-course/en/unit2/smolagents/code_agents)
The party idea is a 'villain masquerade' theme") As you can see, we've created a CodeAgent with several tools that 
enhance the agent's functionality, turning it into the ultimate party planner ready to share with the community! 🎉

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the search results, I can compile a playlist that includes elements from classical music, dark   
cabaret, opera, and electronic music to create a villainous masquerade vibe.                                       
                                                                                                                   
Here's a suggested playlist for a villain masquerade party at Wayne's mansion:                                     
                                                                                                                   
1.  **Classical:**                                                                                                 
    *   "Lacrimosa" - Mozart                                                                                       
    *   "Toccata and Fugue in D Minor" - Bach                                                                      
    *   "Dies Irae" - Verdi (from Requiem)                                                                         
    *   "Night on Bald Mountain" - Mussorgsky                                                                      
    *   "Danse Macabre" - Saint-Saëns                                                                              
2.  **Opera:**                                                                                                     
    *   "Queen of the Night Aria" - Mozart (from The Magic Flute)                                                  
    *   "Habanera" - Bizet (from Carmen)                                                                           
    *   "Nessun Dorma" - Puccini (from Turandot)                                                                   
3.  **Dark Cabaret:**                                                                                              
    *   Anything by The Dresden Dolls                                                                              
    *   Anything by Voltaire                                                                                       
    *   "Missed Me" - The Rasputina                                                                                
4.  **Electronic/Industrial:**                                                                                     
    *   Anything by HEALTH                                                                                         
    *   Anything by Perturbator                                                                                    
    *   "Sweet Dreams (Are Made of This)" - Marilyn Manson (Industrial version)                                    
5.  **Cinematic/Orchestral:**                                                                                      
    *   Soundtracks from Batman movies (Hans Zimmer, Danny Elfman)                                                 
    *[38;2

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("""Suggested Playlist for a Villain Masquerade Party at Wayne's Mansion:                            
                                                                                                                   
  1. Classical:                                                                                                    
      * "Lacrimosa" - Mozart                                                                                       
      * "Toccata and Fugue in D Minor" - Bach                                                                      
      * "Dies Irae" - Verdi (from Requiem)                                                                         
      * "Night on Bald Mountain" - Mussorgsky                                                                      
      * "Danse Macabre" - Saint-Saëns                                                                              
  2. Opera:                                                                                                        
      * "Queen of the Night Aria" - Mozart (from The Magic Flute)                                                  
      * "Habanera" - Bizet (from Carmen)                                                                           
      * "Nessun Dorma" - Puccini (from Turandot)                                                                   
  3. Dark Cabaret:                                                                                                 
      * Anything by The Dresden Dolls                                                                              
      * Anything by Voltaire                                                                                       
      * "Missed Me" - The Rasputina                                                                                
  4. Electronic/Industrial:                                                                                        
      * Anything by HEALTH                                                                                         
      * Anything by Perturbator                                                                                    
      * "Sweet Dreams (Are Made of This)" - Marilyn Manson (Industrial version)                                    
  5. Cinematic/Orchestral:                                                                                         
      * Soundtracks from Batman movies (Hans Zimmer, Danny Elfman)                                                 
      * "The Imperial March" - John Williams (from Star Wars)                                                      
      * Soundtracks from villainous movie characters""")                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Suggested Playlist for a Villain Masquerade Party at Wayne's Mansion:

1. Classical:
    * "Lacrimosa" - Mozart
    * "Toccata and Fugue in D Minor" - Bach
    * "Dies Irae" - Verdi (from Requiem)
    * "Night on Bald Mountain" - Mussorgsky
    * "Danse Macabre" - Saint-Saëns
2. Opera:
    * "Queen of the Night Aria" - Mozart (from The Magic Flute)
    * "Habanera" - Bizet (from Carmen)
    * "Nessun Dorma" - Puccini (from Turandot)
3. Dark Cabaret:
    * Anything by The Dresden Dolls
    * Anything by Voltaire
    * "Missed Me" - The Rasputina
4. Electronic/Industrial:
    * Anything by HEALTH
    * Anything by Perturbator
    * "Sweet Dreams (Are Made of This)" - Marilyn Manson (Industrial version)
5. Cinematic/Orchestral:
    * Soundtracks from Batman movies (Hans Zimmer, Danny Elfman)
    * "The Imperial March" - John Williams (from Star Wars)
    * Soundtracks from villainous movie characters

[Step 3: Duration 4.02 seconds| Input tokens: 15,804 | Output tokens: 159]

'Suggested Playlist for a Villain Masquerade Party at Wayne\'s Mansion:\n\n1. Classical:\n    * "Lacrimosa" - Mozart\n    * "Toccata and Fugue in D Minor" - Bach\n    * "Dies Irae" - Verdi (from Requiem)\n    * "Night on Bald Mountain" - Mussorgsky\n    * "Danse Macabre" - Saint-Saëns\n2. Opera:\n    * "Queen of the Night Aria" - Mozart (from The Magic Flute)\n    * "Habanera" - Bizet (from Carmen)\n    * "Nessun Dorma" - Puccini (from Turandot)\n3. Dark Cabaret:\n    * Anything by The Dresden Dolls\n    * Anything by Voltaire\n    * "Missed Me" - The Rasputina\n4. Electronic/Industrial:\n    * Anything by HEALTH\n    * Anything by Perturbator\n    * "Sweet Dreams (Are Made of This)" - Marilyn Manson (Industrial version)\n5. Cinematic/Orchestral:\n    * Soundtracks from Batman movies (Hans Zimmer, Danny Elfman)\n    * "The Imperial March" - John Williams (from Star Wars)\n    * Soundtracks from villainous movie characters'